Import necessary libraries.

In [217]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import re


Make a GET request to the URL and extract the HTML content

In [218]:
# Define the URL we want to access
url = 'https://www.wunderground.com/dashboard/pws/IFLETC15/table/2023-03-10/2023-03-10/monthly'

# Use the requests library to get the content of the URL
response = requests.get(url)

# Extract the content of the response object
html_content = response.content


Parse the HTML content using BeautifulSoup

In [219]:
soup = BeautifulSoup(html_content, 'html.parser')


Find the table you want to scrape using the CSS selector


In [220]:
# Select the table element that contains the weather data
table = soup.select_one('#main-page-content > div > div > div > lib-history > div.history-tabs > lib-history-table > div > div > div > table')

Extract the table headers

In [221]:
# Create an empty list for the headers.
headers = []
# This code block extracts the headers of the table from the second row of the table's header.
for th in table.select('thead tr')[1].select('th'):
    if th.has_attr('colspan'):
        headers += [th.text.strip()] * int(th['colspan'])
    else:
        headers.append(th.text.strip())
# Extract the table rows
rows = table.select('tbody tr')

Create an empty list to store the data and create the initial data frame.

In [222]:
data_list = []
# Extracting data from each row of the table and appending it to a list.
for row in rows:
    cells = row.select('td')
    data = [cell.text.strip() for cell in cells]
    data_list.append(data)

# Create a data frame
df = pd.DataFrame(data_list, columns=headers)


Rename the column names, to be identifiable by the measure. 

In [223]:
# Create new column names
column_names = ['Date', 'High_1', 'Avg_1', 'Low_1', 'High_2', 'Avg_2', 'Low_2', 'High_3', 'Avg_3', 'Low_3', 'High_4', 'Avg_4', 'Low_4', 'High_5', 'Low_5', 'Sum']
df.columns = column_names

# Rename the columns 
df = df.rename(columns={
    'High_1': 'MAX_temp',
    'Avg_1': 'AVG_temp',
    'Low_1': 'MIN_temp',
    'High_2': 'MAX_dew_point',
    'Avg_2': 'AVG_dew_point',
    'Low_2': 'MIN_dew_point',
    'High_3': 'MAX_humidity',
    'Avg_3': 'AVG_humidity',
    'Low_3': 'MIN_humidity',
    'High_4': 'MAX_wind_speed',
    'Avg_4': 'AVG_wind_speed',
    'Low_4': 'MIN_wind_speed',
    'High_5': 'MAX_pressure',
    'Low_5': 'MIN_pressure',
    'Sum': 'SUM_precipitation'
})


Convert the date from MM/DD/YYYY to DD/MM/YYYY (optional step, dependant on desired format)

In [224]:
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')
# Convert the Date column to the desired format
df['Date'] = df['Date'].dt.strftime('%d/%m/%Y')

Create a function to convert from Fahrenheit to Celsius  (optional step, dependant on desired format)

In [225]:
def convert_temp(temp_str):
    temp_f = float(temp_str.split('\xa0')[0])
    temp_c = (temp_f - 32) * 5 / 9
    return round(temp_c, 1)

# Apply the function to the temperature columns
df['MAX_temp'] = df['MAX_temp'].apply(convert_temp)
df['AVG_temp'] = df['AVG_temp'].apply(convert_temp)
df['MIN_temp'] = df['MIN_temp'].apply(convert_temp)
df['MAX_dew_point'] = df['MAX_dew_point'].apply(convert_temp)
df['AVG_dew_point'] = df['AVG_dew_point'].apply(convert_temp)
df['MIN_dew_point'] = df['MIN_dew_point'].apply(convert_temp)



Create a function to trim the humidity field and convert it to a float %

In [226]:
def extract_humidity(humidity_str):
    humidity_str = humidity_str.split(' ')[0]
    humidity_str = ''.join(filter(str.isdigit, humidity_str))
    return float(humidity_str) / 100

# Apply the function to the humidity columns
df['MAX_humidity'] = df['MAX_humidity'].apply(extract_humidity)
df['AVG_humidity'] = df['AVG_humidity'].apply(extract_humidity)
df['MIN_humidity'] = df['MIN_humidity'].apply(extract_humidity)



Create a function to convert from miles per hour, to kilometres per hour. (optional step, dependant on desired format)

In [227]:
def convert_wind_speed(wind_speed_str):
    wind_speed_str = wind_speed_str.replace(u'\xa0', ' ')
    wind_speed_mph = float(wind_speed_str.split(' ')[0])
    wind_speed_kmph = wind_speed_mph * 1.60934
    return round(wind_speed_kmph, 1)

# Apply the function to the wind speed columns. 
df['MAX_wind_speed'] = df['MAX_wind_speed'].apply(convert_wind_speed)
df['AVG_wind_speed'] = df['AVG_wind_speed'].apply(convert_wind_speed)
df['MIN_wind_speed'] = df['MIN_wind_speed'].apply(convert_wind_speed)


Create a function to convert from  inches of mercury (inHg) to hectopascals (hPa).

In [228]:
def convert_pressure(pressure_str):
    # Remove non-numeric characters from the pressure string
    pressure_num_str = re.sub(r'[^\d\.]', '', pressure_str)
    # Convert the numeric string to a float
    pressure_in = float(pressure_num_str) 
    # Convert pressure from inches of mercury to hectopascals
    pressure_hpa = pressure_in * 33.86389
    return round(pressure_hpa, 2)

# Apply the function to the pressure columns. 
df['MAX_pressure'] = df['MAX_pressure'].apply(convert_pressure)
df['MIN_pressure'] = df['MIN_pressure'].apply(convert_pressure)

Create a function to convert from inches to millimetres. (optional step, dependant on desired format)

In [229]:
def convert_precipitation(precip_str):
    precip_num_str = re.sub(r'[^\d\.]', '', precip_str)
    precip_in = float(precip_num_str)
    precip_mm = precip_in * 25.4
    return round(precip_mm, 2)

# Apply the function to the precipitation columns.
df['SUM_precipitation'] = df['SUM_precipitation'].apply(lambda x: x.strip().split(' ')[0]).apply(convert_precipitation)


Write to a csv 

In [230]:
df.to_csv('daks_weather_data.csv', index=False)
